In [2]:
from pathlib import Path
import numpy as np
import cv2 as cv
import pandas as pd
import yaml
import random, time

In [6]:
# Define paths

ROOT = Path(".")
CALIB = ROOT / "calibration"
SEQ   = CALIB / "mav0"
CAM0  = SEQ / "cam0"
CAM1  = SEQ / "cam1"

CSV_NAME   = "data.csv"             # contains '#timestamp [ns],filename'
IMG_DIR    = "data"                 # images live here
TARGET_YML = CALIB / "checkerboard_7x6.yaml"

OUT = ROOT / "calibration_output"
OUT.mkdir(parents=True, exist_ok=True)

print("cam0:", CAM0.resolve())
print("cam1:", CAM1.resolve())
print("target:", TARGET_YML.resolve())


cam0: /Users/natalia/Documents/LTU_Master/Computer_vision/Lab4/calibration/mav0/cam0
cam1: /Users/natalia/Documents/LTU_Master/Computer_vision/Lab4/calibration/mav0/cam1
target: /Users/natalia/Documents/LTU_Master/Computer_vision/Lab4/calibration/checkerboard_7x6.yaml


In [7]:
# Load checkerboard parameters

with open(TARGET_YML, "r") as f:
    cfg = yaml.safe_load(f)

cols = int(cfg["targetCols"])              # internal corners across columns
rows = int(cfg["targetRows"])              # internal corners across rows
sx   = float(cfg.get("colSpacingMeters", 0.06))  # meters
sy   = float(cfg.get("rowSpacingMeters", 0.06))  # meters

print(f"Checkerboard: {cols} x {rows} internal corners, square {sx} x {sy} m")

# Read CSVs
df0 = pd.read_csv(CAM0 / CSV_NAME)
df1 = pd.read_csv(CAM1 / CSV_NAME)

fn_col0 = [c for c in df0.columns if "filename" in c][0]
fn_col1 = [c for c in df1.columns if "filename" in c][0]

# Pair frames by common filenames (EuRoC-style datasets use matching names/timestamps)
set0 = set(df0[fn_col0].astype(str))
set1 = set(df1[fn_col1].astype(str))
common = sorted(list(set0.intersection(set1)))
print(f"Paired frames: {len(common)}")

# Build absolute paths for pairs
pairs = [(CAM0/IMG_DIR/f, CAM1/IMG_DIR/f) for f in common]
len(pairs), pairs[:2]


Checkerboard: 6 x 7 internal corners, square 0.06 x 0.06 m
Paired frames: 2376


(2376,
 [(PosixPath('calibration/mav0/cam0/data/1403708994187836928.png'),
   PosixPath('calibration/mav0/cam1/data/1403708994187836928.png')),
  (PosixPath('calibration/mav0/cam0/data/1403708994237837056.png'),
   PosixPath('calibration/mav0/cam1/data/1403708994237837056.png'))])

In [8]:
# Prepare the 3D object points for one board view (Z=0 plane)

objp = np.zeros((rows*cols, 3), np.float32)
gx, gy = np.meshgrid(np.arange(cols), np.arange(rows))
objp[:,0] = gx.flatten() * sx
objp[:,1] = gy.flatten() * sy
# objp[:,2] = 0 already

# Settings for detection and subpixel refinement
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 1e-3)
flags = cv.CALIB_CB_ADAPTIVE_THRESH | cv.CALIB_CB_NORMALIZE_IMAGE | cv.CALIB_CB_FAST_CHECK

# Accumulators
objpoints  = []   # list of (N,3)
imgpoints0 = []   # list of (N,1,2) for cam0
imgpoints1 = []   # list of (N,1,2) for cam1
imsize0 = None
imsize1 = None

kept = 0
for p0, p1 in pairs:
    img0 = cv.imread(str(p0), cv.IMREAD_GRAYSCALE)
    img1 = cv.imread(str(p1), cv.IMREAD_GRAYSCALE)
    if img0 is None or img1 is None:
        continue

    if imsize0 is None:
        imsize0 = (img0.shape[1], img0.shape[0])  # (w,h)
    if imsize1 is None:
        imsize1 = (img1.shape[1], img1.shape[0])

    ret0, corners0 = cv.findChessboardCorners(img0, (cols, rows), flags=flags)
    ret1, corners1 = cv.findChessboardCorners(img1, (cols, rows), flags=flags)
    if not (ret0 and ret1):
        continue

    # subpixel refinement
    corners0 = cv.cornerSubPix(img0, corners0, (11,11), (-1,-1), criteria)
    corners1 = cv.cornerSubPix(img1, corners1, (11,11), (-1,-1), criteria)

    objpoints.append(objp.copy())
    imgpoints0.append(corners0)
    imgpoints1.append(corners1)
    kept += 1

print(f"Valid paired detections: {kept}")
print("imsize cam0:", imsize0, " | imsize cam1:", imsize1)
assert kept >= 10, "Need at least ~10 good views for a stable calibration."


Valid paired detections: 1649
imsize cam0: (752, 480)  | imsize cam1: (752, 480)


In [16]:
# Calibrate cam0 intrinsics

# how many paired views to use
N = 1649      # try 80 first
SEED = 40   # for reproducibility

random.seed(SEED)
idx = random.sample(range(len(objpoints)), k=min(N, len(objpoints)))

obj_sub  = [objpoints[i]  for i in idx]
img0_sub = [imgpoints0[i] for i in idx]
img1_sub = [imgpoints1[i] for i in idx]

print(f"Using {len(obj_sub)} views out of {len(objpoints)} total")


# cam0 intrinsics
rms0, K0, dist0, rvecs0, tvecs0 = cv.calibrateCamera(
    objectPoints=obj_sub,
    imagePoints=img0_sub,
    imageSize=imsize0,
    cameraMatrix=None,
    distCoeffs=None
)
print("cam0 RMS:", rms0)
print("cam0 K:\n", K0)
print("cam0 dist:", dist0.ravel())

# cam1 intrinsics
rms1, K1, dist1, rvecs1, tvecs1 = cv.calibrateCamera(
    objectPoints=obj_sub,
    imagePoints=img1_sub,
    imageSize=imsize1,
    cameraMatrix=None,
    distCoeffs=None
)
print("\ncam1 RMS:", rms1)
print("cam1 K:\n", K1)
print("cam1 dist:", dist1.ravel())


Using 1649 views out of 1649 total
cam0 RMS: 0.5962455234274932
cam0 K:
 [[461.27295302   0.         361.31911058]
 [  0.         459.75826361 247.61789372]
 [  0.           0.           1.        ]]
cam0 dist: [-3.11914860e-01  1.28378549e-01  4.36021106e-05  9.41519938e-05
 -2.79416919e-02]

cam1 RMS: 0.7758022806213101
cam1 K:
 [[459.12670346   0.         373.08564114]
 [  0.         457.74340931 253.99528538]
 [  0.           0.           1.        ]]
cam1 dist: [-3.11642114e-01  1.31979842e-01 -2.19971616e-04  4.06962852e-04
 -3.14582706e-02]


In [19]:
# Save instrinsc parameters to disk (NPZ + YAML)
OUT = Path("calibration_output")
OUT.mkdir(parents=True, exist_ok=True)

def save_intrinsics(base, K, dist, imsize, rms, name):
    # NPZ
    np.savez(base.with_suffix(".npz"),
             K=K, dist=dist, width=int(imsize[0]), height=int(imsize[1]),
             rms=float(rms), camera=name)
    # YAML
    yml = {
        "camera": name,
        "image_width": int(imsize[0]),
        "image_height": int(imsize[1]),
        "rms_reprojection_error": float(rms),
        "camera_matrix": {"rows": 3, "cols": 3, "data": K.tolist()},
        "distortion_coefficients": {"rows": 1, "cols": len(dist.ravel()), "data": dist.ravel().tolist()},
        "model": "pinhole",
    }
    with open(base.with_suffix(".yaml"), "w") as f:
        yaml.safe_dump(yml, f, sort_keys=False)

# Store cam0 and cam1 intrinsics
save_intrinsics(OUT / "cam0_intrinsics", K0, dist0, imsize0, rms0, "cam0")
save_intrinsics(OUT / "cam1_intrinsics", K1, dist1, imsize1, rms1, "cam1")

print("Saved to:", OUT.resolve())


Saved to: /Users/natalia/Documents/LTU_Master/Computer_vision/Lab4/calibration_output


In [21]:
# Store corner detections to disk for later use

OUT = Path("calibration_output")
OUT.mkdir(parents=True, exist_ok=True)

np.savez(OUT / "corner_data.npz",
         objpoints=objpoints,      # list of (Ncorners,1,2) per view (for imgpoints)
         imgpoints0=imgpoints0,
         imgpoints1=imgpoints1,
         imsize0=imsize0,          # (width, height)
         imsize1=imsize1)

print("Corner data saved to:", (OUT / "corner_data.npz").resolve())
print("Views saved:", len(objpoints))


Corner data saved to: /Users/natalia/Documents/LTU_Master/Computer_vision/Lab4/calibration_output/corner_data.npz
Views saved: 1649


In [5]:
# Stereo extrinsics (R,T) with timing; save NPZ+YAML

OUT = Path("calibration_output")

# Load intrinsics saved earlier
d0 = np.load(OUT / "cam0_intrinsics.npz")
d1 = np.load(OUT / "cam1_intrinsics.npz")
K0, dist0 = d0["K"], d0["dist"]
K1, dist1 = d1["K"], d1["dist"]
imsize0 = (int(d0["width"]), int(d0["height"]))
imsize1 = (int(d1["width"]), int(d1["height"]))
assert imsize0 == imsize1, "cam0/cam1 image sizes must match"

# Load detected corners saved earlier
cd = np.load(OUT / "corner_data.npz", allow_pickle=True)
objpoints  = list(cd["objpoints"])
imgpoints0 = list(cd["imgpoints0"])
imgpoints1 = list(cd["imgpoints1"])
print(f"Stereo pairs to use: {len(objpoints)}")

# Fix intrinsics; estimate only R,T (and E,F)
flags = cv.CALIB_FIX_INTRINSIC
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 80, 1e-5)

# Run & time
t0 = time.perf_counter()
rms_st, K0_, dist0_, K1_, dist1_, R, T, E, F = cv.stereoCalibrate(
    objectPoints=objpoints,
    imagePoints1=imgpoints0,
    imagePoints2=imgpoints1,
    cameraMatrix1=K0, distCoeffs1=dist0,
    cameraMatrix2=K1, distCoeffs2=dist1,
    imageSize=imsize0,
    flags=flags,
    criteria=criteria
)
elapsed = time.perf_counter() - t0

# Report
baseline = float(np.linalg.norm(T))
print(f"\nStereo calibration done in {elapsed:.1f} s  ({elapsed/60:.2f} min)")
print(f"Stereo RMS: {rms_st:.6f}")
print("R (cam0 -> cam1):\n", R)
print("T (cam0 -> cam1):\n", T.ravel())
print(f"Baseline |T|: {baseline:.6f} (units = checkerboard units)")

# Save results
np.savez(OUT / "stereo_extrinsics.npz", R=R, T=T, E=E, F=F, rms=rms_st)
with open(OUT / "stereo_extrinsics.yaml", "w") as f:
    yaml.safe_dump({
        "stereo_rms": float(rms_st),
        "rotation_matrix": R.tolist(),
        "translation_vector": T.ravel().tolist(),
        "baseline": baseline,
        "essential_matrix": E.tolist(),
        "fundamental_matrix": F.tolist(),
        "note": "R,T map 3D points from cam0 frame to cam1 frame"
    }, f, sort_keys=False)

print("\nSaved: stereo_extrinsics.(npz|yaml) ->", OUT.resolve())


Stereo pairs to use: 1649

Stereo calibration done in 12618.4 s  (210.31 min)
Stereo RMS: 5.223422
R (cam0 -> cam1):
 [[ 0.99999369  0.0027334   0.00226879]
 [-0.00276694  0.99988494  0.01491455]
 [-0.00222776 -0.01492074  0.9998862 ]]
T (cam0 -> cam1):
 [-0.10944116  0.00071261 -0.00093779]
Baseline |T|: 0.109447 (units = checkerboard units)

Saved: stereo_extrinsics.(npz|yaml) -> /Users/natalia/Documents/LTU_Master/Computer_vision/Lab4/calibration_output
